In [44]:
#from components.controller.bluetooth_controller import ServerForAppArduinoBlueControlV2

from components.gyroscope.gyroscope import AngularSpeedControlV2
from components.two_wheels import TwoWheelsV2

from data_collection.data_collection import LoggerSet, Logger
from multiprocessing import Manager
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
#s.terminate()
logger_set = LoggerSet('../log/temp', overwrite_ok=True)


In [77]:
w_p.terminate()
imu_p.terminate()

In [78]:
from ctypes import c_double
manager = Manager()

speed = manager.Value(c_double, 0)
angular_velocity = manager.Value(c_double, 0)

_, w_starter = TwoWheelsV2.create(
    manager, 
    interval=1/50, 
    entry_kwargs=dict(
        left_pin=33, 
        right_pin=32, 
        dir_pins=(16, 18), 
        logger_set=logger_set, 
        name='TwoWheelsV2'
    )
)

imu_out, imu_starter = AngularSpeedControlV2.create(
    manager, 
    interval=1/50, 
    entry_kwargs=dict(
        logger_set=logger_set, 
        name='AngularSpeedControlV2'
    )
)



In [67]:
imu_out

[<ValueProxy object, typeid 'Value' at 0x7f49459130>,
 <ValueProxy object, typeid 'Value' at 0x7f49459100>]

In [79]:
w_p = w_starter(imu_out)
imu_p = imu_starter([angular_velocity, speed])


/home/kawa/projects/car1/src/components/two_wheels.py:15: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:69: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[0], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:70: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[1], GPIO.OUT) #type: ignore


In [80]:
import time
def timeout_callback(func, sec, check_interval=0.1, *args, **kwargs):
    start = time.monotonic()
    while (time.monotonic() - start) < sec:
        time.sleep(check_interval)
    func(*args, **kwargs)

speed.value=100

def f(x):
    x.value=0
timeout_callback(f, 4, 0.1, speed)